# Nasa Predictive Maintenance (RUL)
## Importing & Preprocessing
Aircraft components are susceptible to degradation, which affects directly their reliability and performance. This notebook is directted to predict the remaining useful life (RUL) based on the entire life cycle data in order to provide the necessary maintenance behavior. 

In [ ]:
# -- Standard Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# -- Model --
# Later
# -- Error Metrics --
# -- Preprocessing --
from sklearn.pipeline import make_pipeline
np.random.seed(17)

We have to define feature names

In [ ]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names

Let's import train and validation data, FD0001 subset corresponds to HPC failure of the engine, note: train data will be splitted to train/test sets in the modeling part

In [ ]:
from pathlib import Path

data_path = Path("..") / "data" / "CMaps"


df_train = pd.read_csv(data_path / "train_FD001.txt",
                       sep=r'\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv( data_path / "test_FD001.txt",
                      sep=r'\s+', header=None, index_col=False, names=col_names)
y_valid = pd.read_csv(data_path /"RUL_FD001.txt",
                     sep=r'\s+', header=None, names=['RUL'])
print(f"Test Shape: {df_test.shape}\nTrain Shape:{df_train.shape}\nRUL Labels: {y_valid.shape}")
print(f"Percentage of the test-set: {len(df_test)/(len(df_train)+len(df_test))*100:.3f}%")

With `r's+'` we make raw strings (this treats backslashes as literal characters)

In [ ]:
df_train.describe().transpose()

In [ ]:
null_comparison = pd.concat(
    [df_train.isnull().sum(), df_test.isnull().sum()],
    axis=1,
    keys=['Train_Nulls', 'Test_Nulls']
)
if (null_comparison==0).all().all():
    print("No missing values in either Train or Test data")
else:
    display(null_comparison)

In [ ]:
df_train.loc[:, ['unit_number', 'time_cycles']].describe()

The average engine breaks between 199 and 206 cycles, however the standard deviation of 46 cycles is rather big. 

In [ ]:
df_train.loc[:, 's_1':].describe().transpose()

## Data Visualization & Feature Engineering
Max-Time cycle found for each unit, let's see the maximum time cycle of each unit

In [ ]:
max_time_cycles = df_train[index_names].groupby('unit_number').max() # Groups data by engine ID (max time_cycles)
plt.figure(figsize=(20, 50))
ax = max_time_cycles['time_cycles'].plot(kind='barh', width=0.8, 
                                         stacked=True) # Select just time_cycles 
plt.title('Turbofan Engines LifeTime', size=20)
plt.xlabel('Time Cycle', size=20)
plt.xticks(size=15)
plt.ylabel('Unit', size=20)
plt.yticks(size=15)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
sns.displot(max_time_cycles['time_cycles'], kde=True, bins=20,
            height=6, aspect=3)
plt.xlabel('Max Time Cycle');

The maximum time cycles an engine can achieve is between 190 and 219 before _failure_

### Feature: Remaining Useful Lifecycle (RUL)
This is the remaining time cycles for each unit before it fails

In [ ]:
def add_RUL(df, unit_col='unit_number', cycle_col='time_cycles', 
            rul_col='RUL', clip_min=None):
    max_cycles = df.groupby(unit_col)[cycle_col].transform('max')
    df[rul_col] = max_cycles - df[cycle_col]
    if clip_min is not None:
        df[rul_col] = df[rul_col].clip(lower=clip_min)
    return df
df_train = add_RUL(df_train)
df_train[['unit_number','RUL']]

In [ ]:
max_RUL = df_train.groupby('unit_number').max().reset_index() # Reset index keep unit_number as column
max_RUL.head()

In [ ]:
corr = df_train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool)) # this removes redundancy
plt.figure(figsize=(12, 10))
cmap = sns.diverging_palette(230, 10, as_cmap=True)

sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    annot=True,          
    fmt=".1f",         
    annot_kws={"size": 8},
    vmax=1.0,          
    center=0,
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": 0.5}
)
plt.title("Feature Correlation Matrix (Upper Triangle)", pad=20)
plt.xticks(rotation=45, ha="right")  # Rotate x-labels
plt.tight_layout()
plt.show()

In [ ]:
numeric_df = df_train.select_dtypes("number")  # Only numeric columns
sns.clustermap(numeric_df, cmap="viridis", figsize=(8, 8))
plt.show()

WE have features highly correlated (multicollinearity), features with low correlation may be dropped later.

1. Can we effciently predict the RUL time for the engine (error significance)?
2. Which features are the most important for predicting the failure of the turbofan engine?
3. Does adding historical data improve our model?
4. How can we turn our problem to a classification one?

First, let's match each sensor with it's real signifcation

In [ ]:
Sensor_dictionary={}
dict_list=[ "(Fan inlet temperature) (◦R)",
"(LPC outlet temperature) (◦R)",
"(HPC outlet temperature) (◦R)",
"(LPT outlet temperature) (◦R)",
"(Fan inlet Pressure) (psia)",
"(bypass-duct pressure) (psia)",
"(HPC outlet pressure) (psia)",
"(Physical fan speed) (rpm)",
"(Physical core speed) (rpm)",
"(Engine pressure ratio(P50/P2)",
"(HPC outlet Static pressure) (psia)",
"(Ratio of fuel flow to Ps30) (pps/psia)",
"(Corrected fan speed) (rpm)",
"(Corrected core speed) (rpm)",
"(Bypass Ratio) ",
"(Burner fuel-air ratio)",
"(Bleed Enthalpy)",
"(Required fan speed)",
"(Required fan conversion speed)",
"(High-pressure turbines Cool air flow)",
"(Low-pressure turbines Cool air flow)" ]
i=1
for x in dict_list :
    Sensor_dictionary['s_'+str(i)]=x
    i+=1
Sensor_dictionary

A low pressure compressor (LPC) and high pressure compressor (HPC) supply compressed high temperature, high pressure gases to the combustor. Low pressure turbine (LPT) can decelerate and pressurize air to improve the chemical energy conversion efficiency of aviation kerosene. High pressure turbines (HPT) generate mechanical energy by using high temperature and high pressure gas strike turbine blades. Low-pressure rotor (N1), high-pressure rotor (N2), and nozzle guarantee the combustion efficiency of the engine.

Let's plot the evolution of features (sensors) along with the evolution with RUL

In [ ]:
def plot_signal(df, Sensor_dic, signal_name):
    plt.figure(figsize=(13,5))
    for i in df['unit_number'].unique():
        if (i % 10 == 0):   #For a better visualisation, we plot the sensors signals of 20 units only
            plt.plot('RUL', signal_name, data=df[df['unit_number']==i].rolling(10).mean())

    plt.xlim(250, 0)  # reverse the x-axis so RUL counts down to zero
    plt.xticks(np.arange(0, 300, 25))
    plt.ylabel(Sensor_dic[signal_name])
    plt.xlabel('Remaining Useful Life')
    plt.show()

In [ ]:
for i in range(1,22):
    try:
        plot_signal(df_train, Sensor_dictionary,'s_'+str(i))
    except:
        pass

Sensors that are constant, don't influence the RUL $\rightarrow$ we can drop those ones.

In [ ]:
for x in sensor_names:
    plt.figure(figsize=(12,6))
    plt.boxplot(df_train[x])
    plt.title(x)
    plt.show()

Because there are sensors that are constant (1, 5, 10, 16, 18, 19) and some have outliers, we can scale the data.

In [ ]:
df_train.loc[:, 's_1':].describe().T